# Discretization for Radial Concentrations

- Let $\oplus$ denote the concatenation function. 
- Let $\textbf{rmesh} = \bigoplus_{r=r_0=50\mu m}^{r_\text{max}=1000\mu m}r \equiv \bigoplus_{i=0}^{N-1}r_i $ denote the 1D array of radial distances from the aggregate.  Note that $\text{rmesh}$ is an array of length N.
- Let $c(r,t)$ denote the cAMP concentration at radial distance $r$ at time $t$ in nM.
- Let $pde(r,t)$ denote the PDE concentration at radial distance $r$ at time $t$ in nM.
- Let $F_0(t)\in[0,200]$ nM/s be the rate of change in cAMP concentration at the aggregate at time $t$.
- Let $dt = 0.005s$ be the size of one time step and $dr=1\mu m$ be the spatial resolution of $\textbf{rmesh}$.

Then, we may compute $c(r,t+dt) = c(r,t) + dt \cdot \frac{dc}{dt}\big|_t$, where $\frac{dc}{dt}\big|_t$ is returned by the function `time_step`.

`time_step` is effectively defined from the following steps:

__step 1:__ compute the face terms
-  $\textbf{dface}(t) = 0 \oplus \bigg(\bigoplus_{i=0}^{N-2}c(r_{i+1},t)-c(r_i,t)\bigg)\oplus 0$. Note $\textbf{dface}$ has length N+1
-  $\textbf{cp}(t) = c(r_0,t) \oplus \bigg(\bigoplus_{i=0}^{N-1}c(r_i,t)\bigg)\oplus c(r_\text{max},t)$. Note $\textbf{cp}$ has length N+2
- $\textbf{face}(t) = \frac{1}{2}\bigoplus_{i=0}^{N}\text{cp}(r_{i+1},t)+\text{cp}(r_i,t)$. Note face has length N+1

Let $\text{cp}(r_i,t)$, $\text{face}(r_i,t)$, and $\text{dface}(r_i,t)$ denote the $i^\text{th}$ entry of  $\textbf{cp}(t)$, $\textbf{face}(t)$, and $\textbf{dface}(t)$, respectively.

__step 2:__ compute the transient terms
- $\textbf{term1}(t) = \frac{D}{dr}\bigoplus_{i=0}^{N-1}\frac{1}{r_i}\Big(\text{face}(r_{i+1},t)-\text{face}(r_i,t)\Big)$
- $\textbf{term2}(t) = \frac{D}{dr^2}\bigoplus_{i=0}^{N-1}\text{dface}(r_{i+1},t)-\text{dface}(r_i,t)$
- $\textbf{termDEG}(t) = \frac{D}{dr}\bigoplus_{i=0}^{N-1}\text{face}(r_{i+1},t)-\text{face}(r_i,t)$
- $\textbf{termBC}(t) = F_0\oplus\bigoplus_{i=1}^{N-1}0$

__step 3:__ add the transient terms
Then, 
$$\frac{dc}{dt}\Big|_t = \textbf{term1}(t) + \textbf{term2}(t) + \textbf{termDEG}(t) + \textbf{termBC}(t)$$
Is the vector of rates of change of cAMP concentration at time $t$.

In [1]:
import numpy as np

In [2]:
def time_step(c, pde, rmesh, D, kPDE, dr, fluxLeft, fluxRight=0):
    '''returns d[cAMP]/dt = the rate of change of the cAMP field c
    rmesh is the 1D radial mesh,
    c = 1D array of cAMP concentrations
    pde = 1D array of PDE concentrations
    D = diffusion coefficient of cAMP
    kPDE = decay constant of cAMP due to PDE
    dr = spatial resolution, for example 1 (microns)
    fluxLeft is the rate cAMP is being added at the "left" boundary, closest to the center
    fluxRight is he rate cAMP is being added at the "right" boundary, furthest from the center
    explicitely using only current state.'''
    #step one: compute the face terms
    dface = np.hstack([ 0. , np.diff(c) , 0. ])
    cp    = np.hstack([c[0], c, c[-1]])
    face  = cp[1:]*0.5 + cp[:-1]*0.5

    #step two: compute transient terms 
    term1 = D*np.diff(face)/rmesh/dr
    term2 = D*np.diff(dface)/dr**2
    termDEG = -1*kPDE*c*pde
    #calculate boundary term
    termBC = np.hstack([ fluxLeft , 0*c[1:-1] , fluxRight ])

    #step three: add transient terms
    dcdt  = term1 + term2 + termBC + termDEG
    return dcdt